<a href="https://colab.research.google.com/github/TheSmileXiao/TFM/blob/main/code/load_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages and import

In [ ]:
!pip install ultralytics

In [ ]:
import zipfile
import os
import shutil
from ultralytics          import YOLO
from google.colab         import drive
import random
import tqdm

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


## Build YOLO_DATASET + COCO (with all images randomly distributed)

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/yolo_dataset.zip' ## Dataset path
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/coco-seg_5000samples.zip'  ## COCO Dataset path
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
img_dir = '/content/coco-images'
labels_dir = '/content/coco_annotation'
dst_dir = '/content/yolo_dataset'
def add_coco_dataset(img_dir, labels_dir, dst_dir, train_size=0.7, val_size=0.2, test_size=0.1):  ## Merge COCO Dataset into.
  random.seed(42)
  imgs = os.listdir(img_dir)
  random.shuffle(imgs)
  length = len(imgs)

  train_end = int(train_size * length)
  val_end = train_end + int(val_size * length)

  train_imgs = imgs[:train_end]
  val_imgs = imgs[train_end:val_end]
  test_imgs = imgs[val_end:]

  for img in [img for img in train_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/train'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/train'))
  print(f'{train_end} images have been move.')
  for img in [img for img in val_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/val'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/val'))
  print(f'{val_end-train_end} images have been move.')
  for img in [img for img in test_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/test'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/test'))
  print(f'{length-val_end} images have been move.')

add_coco_dataset(img_dir, labels_dir, dst_dir)

3500 images have been move.
1000 images have been move.
500 images have been move.


## Build YOLO_DATASET + COCO (the images are distributed by videos)

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/video_yolo_dataset.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
ruta_archivo_zip = '/content/drive/MyDrive/TFM_Xiao/Anotaciones/coco-seg_5000samples.zip'
ruta_destino = './'
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Extraer el archivo deseado en la ruta de destino
    archivo_zip.extractall(ruta_destino)

In [ ]:
img_dir = '/content/coco-images'
labels_dir = '/content/coco_annotation'
dst_dir = '/content/yolo_dataset'
def add_coco_dataset(img_dir, labels_dir, dst_dir, train_size=0.8, val_size=0.2, test_size=0):
  random.seed(42)
  imgs = os.listdir(img_dir)
  random.shuffle(imgs)
  length = len(imgs)

  train_end = int(train_size * length)
  val_end = train_end + int(val_size * length)

  train_imgs = imgs[:train_end]
  val_imgs = imgs[train_end:val_end]
  test_imgs = imgs[val_end:]

  for img in [img for img in train_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/train'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/train'))
  print(f'{train_end} images have been move.')
  for img in [img for img in val_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/val'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/val'))
  print(f'{val_end-train_end} images have been move.')
  for img in [img for img in test_imgs if img.endswith('.jpg')]:
    shutil.copy(os.path.join(img_dir, img), os.path.join(dst_dir, 'images/test'))
    shutil.copy(os.path.join(labels_dir, f"{img.split('.')[0]}.txt"), os.path.join(dst_dir, 'labels/test'))
  print(f'{length-val_end} images have been move.')

add_coco_dataset(img_dir, labels_dir, dst_dir)

## Model (Fine tuning and train)

In [ ]:
# Load a model
model = YOLO('yolov9c-seg.yaml')  # build a new model from YAML
model = YOLO('yolov9c-seg.pt')  # load a pretrained model (recommended for training)
model.info()

100%|██████████| 53.9M/53.9M [00:00<00:00, 73.7MB/s]

WARNING ⚠️ yolov9c-seg.pt appears to require 'dill', which is not in ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 5.7s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv9c-seg summary: 654 layers, 27897120 parameters, 0 gradients, 159.4 GFLOPs


(654, 27897120, 0, 159.41411839999998)

### Train model, the yaml file was included in the dataset

In [ ]:
model.train(data='/content/yolo_dataset/dataset.yaml', epochs=50, patience=10, imgsz=640, freeze=10, verbose=True, batch=-1, exist_ok=True,
      project='/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models', name='yolo_v9_video_coco/segment')

### Load model retrained

In [ ]:
# model = YOLO('/content/drive/MyDrive/TFM_Xiao/Code/YOLO/save_models/yolo_v9_video_coco/segment/weights/best.pt')

## Resume train

In [ ]:
#model.train(resume=True)

## Save all files in Google Drive

In [ ]:
drive.flush_and_unmount()